# Docker commands

## Working with containers

### `docker ps` - show containers

In [4]:
%%bash
docker run -itd --rm --name test_container python:3.10 &> /dev/null
docker ps
docker stop test_container &> /dev/null

CONTAINER ID   IMAGE         COMMAND     CREATED        STATUS                  PORTS     NAMES
b17984f2b882   python:3.10   "python3"   1 second ago   Up Less than a second             test_container


##### `-a` show all containers

By default, inactive containers are hidden.

In [8]:
%%bash
docker run --name temp_container ubuntu &> /dev/null
echo "========no option -a=============="
docker ps
echo "=========with option -a============="
docker ps -a
docker rm temp_container &> /dev/null

========no option -a==============
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
=========with option -a=============
CONTAINER ID   IMAGE     COMMAND       CREATED                  STATUS                              PORTS     NAMES
ad0439c23847   ubuntu    "/bin/bash"   Less than a second ago   Exited (0) Less than a second ago             temp_container


### `docker history` - show layers of the package

### `docker run` - create the container from image

In [9]:
%%bash
echo "================before start====================="
docker ps -a
echo "===============after start======================"
docker run --name temp_container ubuntu &> /dev/null
docker ps -a
docker rm temp_container

================before start=====================
CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
===============after start======================
CONTAINER ID   IMAGE     COMMAND       CREATED        STATUS                              PORTS     NAMES
cd0f144063c5   ubuntu    "/bin/bash"   1 second ago   Exited (0) Less than a second ago             temp_container
temp_container


##### `-rm` - automatic container removal

After stopping, the container is removed. In the following example there are two containers of ubuntu, one with the `--rm` option and the other without. Both containers stops by default - but only one stays with `STATUS == Exited`.

In [12]:
%%bash
docker run --name no_rm_ubuntu ubuntu
docker run --rm --name ubuntu_with_rm ubuntu
docker ps -a
docker rm no_rm_ubuntu &> /dev/null

CONTAINER ID   IMAGE     COMMAND       CREATED        STATUS                              PORTS     NAMES
7acc3cc0c61c   ubuntu    "/bin/bash"   1 second ago   Exited (0) Less than a second ago             no_rm_ubuntu


##### `--name` - set name for the container

In [2]:
%%bash
docker run --rm --name unbuntu_container -d ubuntu
docker ps
docker stop unbuntu_container

7116dcdaa0a131a8507f9cbe149719498456fe356a23f1ddfc48f65080cfe660
CONTAINER ID   IMAGE     COMMAND       CREATED        STATUS                  PORTS     NAMES
7116dcdaa0a1   ubuntu    "/bin/bash"   1 second ago   Up Less than a second             unbuntu_container
unbuntu_container


##### `-d` - make container not to lock CLI

In [15]:
%%bash
docker run --rm -d --name temp_nginx nginx &> /dev/null
docker ps
docker stop temp_nginx &> /dev/null

CONTAINER ID   IMAGE     COMMAND                  CREATED                  STATUS                  PORTS     NAMES
09175cb8c6d9   nginx     "/docker-entrypoint.…"   Less than a second ago   Up Less than a second   80/tcp    temp_nginx


##### `-i` - interactive mode

Mode that allows to interapt with container.

Unfortunately it is not possible to show how this mode will work in jupyter. So here is a gif of me experimenting with the following command:

`docker run -i --rm --name test_ubuntu ubuntu`

<img src="containers_files/docker -i option.gif"></img>

In the example, the ubuntu container has been started and is stopped by default. The `-i` option causes it to lock the terminal. Extremely interesting that you will be able to send messages to the container and get results (as I showed with `ls` and `exit` commands), but you won't get the description of the user common for Linux terminal.

##### `-t` turn on TTY

TTY is something on backend language. The point here is that in combination with the `-i` option you can make will show you the "description" of the user which was mentioned in the previous section. And the terminal will be blocked waiting for input. But this option don't allow you to send anything to the container.

The following `.gif` show what you will get in case using command `docker run -t --rm --name temp_ubuntu ubuntu`.

<img src="containers_files/docker -t option.gif"></img>

##### `-it` - combination

Both options were discovered above, but they are really often used in combination. So Ubuntu provides full CLI interface with this options.

<img src="containers_files/docker -it option.png"></img>

##### `-e` - environment variables

In [1]:
%%bash
docker run --rm -itd --name e_example -e TEST_VAR="value" ubuntu &> /dev/null
docker exec e_example bash -c "echo \$TEST_VAR"
docker stop e_example &> /dev/null

value


You can also pass the values of the variables of the system you are calling from.

In [3]:
%%bash
docker run --rm -itd --name e_example -e FEDOR_TEST_VAR=$HOME ubuntu &> /dev/null
docker exec e_example bash -c "echo \$FEDOR_TEST_VAR"
docker stop e_example &> /dev/null

/home/fedor


##### **Command on container start**{#sec-docker_run_com_on_start}

In the `docker run` command, after the name of the base image, you can specify a command that will be executed when the container starts. So in the next example I will show you how to run `echo "hello world"` in a new ubuntu container.

In [2]:
%%bash
docker run --rm ubuntu echo "hello world"

hello world


- If you want to run several commands you have to use `bash -c "<command>"`, like in following example:

In [5]:
%%bash
docker run --rm ubuntu bash -c "
    echo \"hello world\"; 
    echo \"hello world2\"
"

hello world
hello world2


- Note that when containers are in interactive mode, the command executed should take input, otherwise it will just leave container anyway, the follwing gif describe how it looks like:
<img src="containers_files/docker command -it option.gif"></img>


In the example, I ran the `echo` command with container starts and showed that there were no containers created - because the container was stopped and removed just after it was started. But immediately I created a new container, but now with the command `bash -c "echo \"hello world\"; bash` - in this case the last command is `bash` which waits for input, so I'm comming back into the container.

### `docker start` - activate exited container

In the next cell, I build a ubuntu container that prints "hello world" when it starts. Like all standard ubuntu containers it quits, which we can see in the `STATUS` of the `ps` command. Then by `docker start` the container is reactivated.

For some reason it only works when I use the `-i` option for `docker start`.

In [16]:
%%bash
cd containers_files
docker build -t test_ubuntu . &> /dev/null
echo "=====First start====="
docker run --name u1 test_ubuntu
echo "=====List of all containers====="
docker ps -a
echo "=====Restart====="
docker start -i u1
docker rm u1 &> /dev/null
docker rmi test_ubuntu &> /dev/null

=====First start=====
hello world
=====List of all containers=====
CONTAINER ID   IMAGE         COMMAND                CREATED        STATUS                              PORTS     NAMES
af533b577c94   test_ubuntu   "echo 'hello world'"   1 second ago   Exited (0) Less than a second ago             u1
=====Restart=====
hello world


### `docker stop` - stop some container

In the following example, I start a container with Ubuntu in such a way that it stays running, which was proven by using `docker ps`. But after using `docker stop` the same container is in the state "Exited".

In [20]:
%%bash
docker run -itd --name test_ubuntu ubuntu &> /dev/null
echo "=====Just started container====="
docker ps -a
docker stop test_ubuntu &> /dev/null
echo "=====Stoped container (STATUS \"Exited\")====="
docker ps -a
docker rm test_ubuntu &> /dev/null

=====Just started container=====
CONTAINER ID   IMAGE     COMMAND       CREATED                  STATUS                  PORTS     NAMES
e175ae336dcb   ubuntu    "/bin/bash"   Less than a second ago   Up Less than a second             test_ubuntu
=====Stoped container (STATUS "Exited")=====
CONTAINER ID   IMAGE     COMMAND       CREATED          STATUS                                PORTS     NAMES
e175ae336dcb   ubuntu    "/bin/bash"   10 seconds ago   Exited (137) Less than a second ago             test_ubuntu


### `docker exec` - run a command in the container

In the next command I start ubuntu conteiner. Then using `docker exec' I run the `ls' command in the container and get a typical Linux result for that command.

In [24]:
%%bash
docker run -itd --rm --name test_ubuntu ubuntu &> /dev/null
docker exec test_ubuntu ls
docker stop test_ubuntu &> /dev/null

bin
boot
dev
etc
home
lib
lib32
lib64
libx32
media
mnt
opt
proc
root
run
sbin
srv
sys
tmp
usr
var


##### `-it` - take control of the container

It looks like the options individually have the same properties as in the `docker run` command. But the important thing is that you can take control of the detached container with this command.<br> 
The following example shows how I can get control in detached ubuntu by running the `bash` command with named options.

<img src="containers_files/docker exec -it.gif"></img>

## Working with images

### `docker pull` - download image from dockerhub.com

It is obligatory to pass on one of the:

- `<image name>:<tag>` (`<tag>` - by default is `latest`);
- `IMAGE ID` - image id.

In [6]:
%%bash
docker images | grep hello-world
echo "===============beforre pull======================"
docker pull hello-world:latest &> /dev/null
echo "===============after pull======================"
docker images | grep hello-world
docker rmi hello-world &> /dev/null

===============beforre pull======================
===============after pull======================
hello-world                latest    9c7a54a9a43c   2 weeks ago     13.3kB


### `docker images` - show list of available images;

In [31]:
%%bash
docker images

REPOSITORY    TAG       IMAGE ID       CREATED         SIZE
hello-world   latest    feb5d9fea6a5   16 months ago   13.3kB


##### `--a`,`--all` - show all images;

By default, so called "intermediate", images are hiden. Nowadays I don't know what "intermediate" images are.

In [32]:
%%bash
docker images --all

REPOSITORY    TAG       IMAGE ID       CREATED         SIZE
hello-world   latest    feb5d9fea6a5   16 months ago   13.3kB


##### `--digests` - show digests;

In [33]:
%%bash
docker images --digests

REPOSITORY    TAG       DIGEST                                                                    IMAGE ID       CREATED         SIZE
hello-world   latest    sha256:aa0cc8055b82dc2509bed2e19b275c8f463506616377219d9642221ab53cf9fe   feb5d9fea6a5   16 months ago   13.3kB


##### `--format` - format output;

Allows output to be formatted using GO templates.

In [34]:
%%bash
docker images --format '{{.Repository}}:{{.Tag}}'

hello-world:latest


##### `--no-trunc` - full output;

In [35]:
%%bash
docker images --no-trunc

REPOSITORY    TAG       IMAGE ID                                                                  CREATED         SIZE
hello-world   latest    sha256:feb5d9fea6a5e9606aa995e879d862b825965ba48de054caab5ef356dc6b3412   16 months ago   13.3kB


##### `-q`, `--quiet` - print only images ids.

In [36]:
%%bash
docker images -q

feb5d9fea6a5


### `docker rmi` - delete images;

It is obligatory to pass on one of the:

- `<image name>:<tag>` (`<tag>` - by default is `latest`);
- `IMAGE ID` - image id.

In [39]:
%%bash
docker images
echo "==========================="
docker rmi hello-world:latest
echo "==========================="
docker images

REPOSITORY    TAG       IMAGE ID       CREATED         SIZE
hello-world   latest    feb5d9fea6a5   16 months ago   13.3kB
Untagged: hello-world:latest
Untagged: hello-world@sha256:aa0cc8055b82dc2509bed2e19b275c8f463506616377219d9642221ab53cf9fe
Deleted: sha256:feb5d9fea6a5e9606aa995e879d862b825965ba48de054caab5ef356dc6b3412
Deleted: sha256:e07ee1baac5fae6a26f30cabfe54a36d3402f96afda318fe0a96cec4ca393359
REPOSITORY   TAG       IMAGE ID   CREATED   SIZE


# `build` command

This is the command used to create a new docker image.

Arguments:

- the last mandatory argument sets the build directory - you should use `.` to set the current directory.
- `-t` - set the name and tag of the image;
- `-f` - set the dockerfile name if it is different from the default `dockerfile`.

In [1]:
%%bash
cd build_files
docker build -t custom_ubuntu:test -f my_first_dockerfile .
docker images | grep custom_ubuntu
docker rmi custom_ubuntu:test

#1 [internal] load .dockerignore
#1 transferring context: 2B done
#1 DONE 0.0s

#2 [internal] load build definition from my_first_dockerfile
#2 transferring dockerfile: 291B done
#2 DONE 0.0s

#3 [internal] load metadata for docker.io/library/ubuntu:latest
#3 DONE 0.0s

#4 [1/3] FROM docker.io/library/ubuntu:latest
#4 DONE 0.0s

#5 [internal] load build context
#5 transferring context: 90B done
#5 DONE 0.0s

#6 [2/3] WORKDIR /home/Dranikf
#6 CACHED

#7 [3/3] COPY script.sh script.sh
#7 CACHED

#8 exporting to image
#8 exporting layers done
#8 writing image sha256:b204cd5e8265b5fb93e193ca74a1bf739406b380dca02e654db0586dbe02f303 done
#8 naming to docker.io/library/custom_ubuntu:test done
#8 DONE 0.0s


custom_ubuntu              test      b204cd5e8265   2 days ago      77.8MB
Untagged: custom_ubuntu:test
Deleted: sha256:b204cd5e8265b5fb93e193ca74a1bf739406b380dca02e654db0586dbe02f303


You should see a difference between building directory and dockerfile. The `dockerfile` is only taken from the building directory if it is not specified in the `-f` option. But the files used for the build will in any case come from the build directory.

In the following example I take the dockerfile from `/build_files` but set the build directory to `/build_files/test_folder`. The dockerfile is designed to copy `scipt.sh` from the building directory, but in named directories `script.sh` is different, so let's check which one is executed.

In [2]:
%%bash
cd build_files
echo "=====tree of the build_files====="
tree

echo
echo

echo "=====script from build_files====="
bash script.sh
echo "=====script from test_folder====="
bash test_folder/script.sh

echo
echo

docker build -t test_ubuntu -f my_first_dockerfile test_folder &> /dev/null
docker run --name test_ubuntu --rm -itd test_ubuntu &> /dev/null
echo "=====message from container====="
docker exec test_ubuntu bash script.sh
docker stop test_ubuntu &> /dev/null
docker rmi test_ubuntu &> /dev/null

=====tree of the build_files=====
.
├── my_first_dockerfile
├── script.sh
├── test_file
└── test_folder
    ├── dockerfile
    └── script.sh

1 directory, 5 files


=====script from build_files=====
It's a script.sh from the images_creation_files
=====script from test_folder=====
It's a script from test_folder


=====message from container=====
It's a script from test_folder


As a result, even though each dockerfile is in the `build_files` folder, the `script.sh` file is taken from the folder specified as the build folder.

Just to check the opposite option. I will use docker file from `test_folder` but set `.` as build directory. So the script result will be from `images_creation_folder`. To know that container was built from dockerfile from `test_folder`, dockerfile will add "hello" to the result of the executed script.

In [3]:
%%bash
cd build_files

echo "=====script from build_files====="
bash script.sh
echo "=====script from test_folder====="
bash test_folder/script.sh

echo
echo

docker build -t test_ubuntu -f test_folder/dockerfile . &> /dev/null
docker run --name test_ubuntu --rm -itd test_ubuntu &> /dev/null
echo "=====message from container====="
docker exec test_ubuntu bash script.sh
docker stop test_ubuntu &> /dev/null
docker rmi test_ubuntu &> /dev/null

=====script from build_files=====
It's a script.sh from the images_creation_files
=====script from test_folder=====
It's a script from test_folder


=====message from container=====
It's a script.sh from the images_creation_files
hello


**Note** even if you have specified a folder as some assembly folder, you must still pass in the `-f` parameter the path relative to the startup folder.

The next two cells show the difference:

- Not specifying a folder for `my_first_dockerfile` will cause an error;

In [35]:
%%bash
docker image build -f my_first_dockerfile -t ubuntu_test build_files &> /dev/null
docker rmi ubuntu_test &> /dev/null

CalledProcessError: Command 'b'docker image build -f my_first_dockerfile -t ubuntu_test image_creation_files &> /dev/null\nsleep 10\ndocker rmi ubuntu_test &> /dev/null\n'' returned non-zero exit status 1.

- Specifying a folder leads to successful completion.

In [36]:
%%bash
docker image build -f build_files/my_first_dockerfile -t ubuntu_test build_files &> /dev/null
docker rmi ubuntu_test &> /dev/null